# Function Calling

In [186]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

## OpenAI 函数回调

### 定义 python 函数

In [251]:
import json
import random

from langchain_core.utils.function_calling import convert_to_openai_tool

def where_cat_is_hiding() -> str:
    """寻找猫的位置"""
    return random.choice(["在床底下", "在书架中"])

def get_items(place: str) -> str:
    """使用这个工具去仔细查看猫躲藏的地方"""
    if "床底" in place:  # For under the bed
        return "发现几只臭袜子和鞋"
    if "书架" in place:  # For 'shelf'
        return "发现一些漫画书、图册和铅笔"
    else:  # 如果智能体决定找其他地方
        return "屋外"

In [252]:
print(json.dumps(convert_to_openai_tool(where_cat_is_hiding), indent=2, ensure_ascii=False))

{
  "type": "function",
  "function": {
    "name": "where_cat_is_hiding",
    "description": "寻找猫的位置",
    "parameters": {
      "type": "object",
      "properties": {},
      "required": []
    }
  }
}


### 使用 Pydantic 补充描述

使用 Pydantic 可以更好的描述工具信息，例如：

- 对参数做单独说明
- 用`...`来说明参数是必须提供的

In [285]:
from langchain_core.pydantic_v1 import BaseModel, Field

class WhereIsCatSchema(BaseModel):
    """找猫"""

    idea: str = Field(description="寻找猫的线索")
    
class GetItemsSchema(BaseModel):
    """看看有什么发现"""

    place: str = Field(..., description="地点的描述")

In [286]:
print(json.dumps(convert_to_openai_tool(WhereIsCatSchema), indent=2, ensure_ascii=False))

{
  "type": "function",
  "function": {
    "name": "WhereIsCatSchema",
    "description": "找猫",
    "parameters": {
      "type": "object",
      "properties": {
        "idea": {
          "description": "寻找猫的线索",
          "type": "string"
        }
      },
      "required": [
        "idea"
      ]
    }
  }
}


### 定义 LangChain 工具

<div class="alert-warning">
<b>注意：</b><br>
OpenAI使用的Tools名称类似于一个函数命名，必须使用下划线或ASCII码，**不能使用中文**！<br>
但描述部份可以使用中文。
</div>

In [287]:
from typing import Any, Type
from langchain_core.tools import BaseTool
import random

class WhereIsCat(BaseTool):
    args_schema: Type[BaseModel] = WhereIsCatSchema
    name: str = "where_is_cat_hidding"
    description: str = "看看猫藏在哪里？"

    def _run(self, idea: str, **kwargs: Any) -> Any:
        return random.choice(["在床底下", "在书架中"])

class GetItems(BaseTool):
    args_schema: Type[BaseModel] = GetItemsSchema
    name: str = "find_place"
    description: str = "仔细翻看猫的藏身之处"

    def _run(self, place: str, **kwargs: Any) -> Any:
        if "床底" in place:  # For under the bed
            return "发现几只臭袜子和鞋"
        if "书架" in place:  # For 'shelf'
            return "发现一些漫画书、图册和铅笔"
        else:  # 如果智能体决定找其他地方
            return "屋外"

In [288]:
# 请注意：我们传入了 Multiply 对象，而不是类
print(json.dumps(convert_to_openai_tool(GetItems()), indent=2, ensure_ascii=False))

{
  "type": "function",
  "function": {
    "name": "find_place",
    "description": "仔细翻看猫的藏身之处",
    "parameters": {
      "type": "object",
      "properties": {
        "place": {
          "description": "地点的描述",
          "type": "string"
        }
      },
      "required": [
        "place"
      ]
    }
  }
}


### 调用 OpenAI

In [289]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [290]:
resp = llm.invoke(
    "猫藏在哪里？",
    tools=[
        convert_to_openai_tool(WhereIsCat()),
        convert_to_openai_tool(GetItems())
    ])
print(resp)

content='' additional_kwargs={'tool_calls': [{'id': 'call_6M4yRwXJRxmIx4fjdb3cDXjU', 'function': {'arguments': '{\n  "idea": "猫可能藏在沙发底下"\n}', 'name': 'where_is_cat_hidding'}, 'type': 'function'}]}


以下方法是等价的：
```python
# bind
llm.bind(tools=[convert_to_openai_tool(Multiply())]).invoke("5的3倍是多少？")
# bind_tools
llm.bind_tools([convert_to_openai_tool(Multiply())]).invoke("5的3倍是多少？")
```

In [291]:
llm.bind(tools=[convert_to_openai_tool(Multiply())]).invoke("5的3倍是多少？")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_sRNYhLTafeMAk514J0ADy7RI', 'function': {'arguments': '{\n  "a": 5,\n  "b": 3\n}', 'name': 'multiply_func'}, 'type': 'function'}]})

## 解析 OpenAI 函数和参数

### 解析为 JSON

In [292]:
from langchain.output_parsers.openai_tools import JsonOutputToolsParser

JsonOutputToolsParser().invoke(resp)

[{'type': 'where_is_cat_hidding', 'args': {'idea': '猫可能藏在沙发底下'}}]

### 提取 JSON 特定键

In [293]:
from langchain.output_parsers.openai_tools import JsonOutputKeyToolsParser

JsonOutputKeyToolsParser(key_name="multiply_func").invoke(resp)

[]

### 使用 Pydantic 校验

In [294]:
from langchain_core.pydantic_v1 import BaseModel, Field, validator

class MultiplyWithValidator(BaseModel):
    """将两个整数相乘。"""

    a: int = Field(..., description="第1个整数")
    b: int = Field(..., description="第2个整数")

    @validator("a")
    def is_a_over(cls, field):
        """检查a是否太大了"""
        if(field > 10):
            raise ValueError("a超过10了")
        return field

In [295]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo").bind_tools([tool])

In [154]:
from langchain.output_parsers.openai_tools import PydanticToolsParser

parser = PydanticToolsParser(tools=[MultiplyWithValidator])

In [155]:
chain = llm | parser

In [175]:
resp = chain.invoke("5的3倍是多少？")
print(resp)

[MultiplyWithValidator(a=5, b=3)]


In [161]:
try:
    chain.invoke("15的3倍是多少？")
except Exception as e:
    print(e)

1 validation error for MultiplyWithValidator
a
  a超过10了 (type=value_error)


### 运行函数

In [180]:
for obj in resp:
    if(isinstance(obj, MultiplyWithValidator)):
        print(multiply(obj.a, obj.b))
    else:
        print("没有工具需要执行")

15


### 小结

如前文所示：GPT模型会在每次检测到需要调用函数时返回这个函数的名称和参数，但需要你自己在代码中去运行工具函数。
通常，你还需要将函数执行结果反馈给GPT，然后生成新的结果。

智能体将自动化上述过程。

## 使用 OpenAI Tool 自定义智能体

### Prompt

In [202]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你正在与我玩一个找猫的游戏",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

### Tool

In [296]:
tools=[
        convert_to_openai_tool(WhereIsCat()),
        convert_to_openai_tool(GetItems())
    ]

### LLM

In [297]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125").bind_tools(tools)

In [299]:
llm.invoke("where is cat hidding?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tqZ3DrsxmMrpTI0Ut3zrv6oQ', 'function': {'arguments': '{"idea":"under the bed"}', 'name': 'where_is_cat_hidding'}, 'type': 'function'}]})

In [300]:
chain = {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    } | prompt | llm

chain.invoke({"input": "猫在哪?", "intermediate_steps": ""})

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qZDAraUxhrHZ5H1eBpU3LgAp', 'function': {'arguments': '{"idea":"猫在哪?"}', 'name': 'where_is_cat_hidding'}, 'type': 'function'}]})

### Agent

In [301]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm
    | OpenAIToolsAgentOutputParser()
)

### AgentExecutor

In [303]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=[WhereIsCat(), GetItems()], verbose=True)

### 运行

In [304]:
agent_executor.invoke({"input": "猫在哪里?"})



> Entering new AgentExecutor chain...

Invoking: `where_is_cat_hidding` with `{'idea': '猫在哪里?'}`


在书架中
Invoking: `find_place` with `{'place': '书架'}`


发现一些漫画书、图册和铅笔猫不在书架上，但我找到了一些漫画书、图册和铅笔。继续寻找猫藏在哪里吗？

> Finished chain.


{'input': '猫在哪里?', 'output': '猫不在书架上，但我找到了一些漫画书、图册和铅笔。继续寻找猫藏在哪里吗？'}

## 使用 create_openai_tools_agent 创建智能体

### 准备

In [305]:
from langchain import hub
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain.prompts import ChatPromptTemplate
from langchain.tools import tool, StructuredTool
from langchain_core.callbacks import Callbacks
from langchain_openai import ChatOpenAI

### Prompt

In [306]:
prompt = hub.pull("hwchase17/openai-tools-agent")
print(prompt.messages) # to see the prompt

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')]


### LLM

In [307]:
model = ChatOpenAI(temperature=0, streaming=False).bind(seed=42)

### Agent

In [308]:
agent = create_openai_tools_agent(model, tools, prompt)

### AgentExecutor

In [310]:
agent_executor = AgentExecutor(agent=agent, tools=[WhereIsCat(), GetItems()], verbose=True)

In [311]:
agent.input_schema()

RunnableSequenceInput(input=None)

### invoke

In [312]:
agent.invoke({"input": "猫呢?", "intermediate_steps":""})

[OpenAIToolAgentAction(tool='where_is_cat_hidding', tool_input={'idea': 'The cat might be hiding under the bed.'}, log="\nInvoking: `where_is_cat_hidding` with `{'idea': 'The cat might be hiding under the bed.'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_IZwjJYNqKry2z270WxiNNQEc', 'function': {'arguments': '{\n  "idea": "The cat might be hiding under the bed."\n}', 'name': 'where_is_cat_hidding'}, 'type': 'function'}]})], tool_call_id='call_IZwjJYNqKry2z270WxiNNQEc')]

In [313]:
tools

[{'type': 'function',
  'function': {'name': 'where_is_cat_hidding',
   'description': '看看猫藏在哪里？',
   'parameters': {'type': 'object',
    'properties': {'idea': {'description': '寻找猫的线索', 'type': 'string'}},
    'required': ['idea']}}},
 {'type': 'function',
  'function': {'name': 'find_place',
   'description': '仔细翻看猫的藏身之处',
   'parameters': {'type': 'object',
    'properties': {'place': {'description': '地点的描述', 'type': 'string'}},
    'required': ['place']}}}]

In [315]:
agent_executor.invoke({"input": "猫呢?"})



> Entering new AgentExecutor chain...

Invoking: `where_is_cat_hidding` with `{'idea': '猫可能躲在沙发底下'}`


在书架中根据线索，猫可能躲在书架中。请仔细检查书架，看看猫是否在那里。

> Finished chain.


{'input': '猫呢?', 'output': '根据线索，猫可能躲在书架中。请仔细检查书架，看看猫是否在那里。'}

In [317]:
# Note: We use `pprint` to print only to depth 1, it makes it easier to see the output from a high level, before digging in.
import pprint

chunks = []

async for chunk in agent_executor.astream(
    {"input": "猫呢?"}
):
    chunks.append(chunk)
    print("------")
    pprint.pprint(chunk, depth=1)



> Entering new AgentExecutor chain...
------
{'actions': [...], 'messages': [...]}

Invoking: `where_is_cat_hidding` with `{'idea': '猫可能躲在沙发底下'}`


在床底下------
{'messages': [...], 'steps': [...]}
猫可能躲在床底下。

> Finished chain.
------
{'messages': [...], 'output': '猫可能躲在床底下。'}
